#DQN

In [20]:
from typing import no_type_check


import numpy as np
from numpy.lib.function_base import append, gradient
import tensorflow as tf
import gym
import os
import datetime
from gym import wrappers
from tensorflow.python.keras.layers.pooling import MaxPool1D

#call with a tf.keras.layer.Input(<whatever>) to see if it works or if I should do the input layer inside

#gotta try tanh vs relu
def dense_nn(layer_sizes, name="defualt"):
    
    #m=tf.keras.models.Sequential()
    x=[]
     # not going to use this so I can add denses whereever tf.keras.layers.Input(shape=inputs.shape)
    


    
    for i,size in enumerate(layer_sizes):

            x.append( tf.keras.layers.Dense(
            size,
            activation=tf.nn.relu ,
            kernel_initializer=tf.keras.initializers.GlorotUniform(),
            name=name+'_l'+str(i))  
            )

    #this mgiht be fucky, yeah is fucky I am subclassing the model not doing functional
    # m=tf.keras.Model(inputs, x)
    # print(x.summary())
    
    return x

class model(tf.keras.Model):
    def __init__(self,num_s, hiddens,num_a, **kwargs):
            super(model,self).__init__()
            self.state_value=None
            self.num_a=num_a
        # if kwargs is None:
        #     self.input_layer= tf.keras.layers.InputLayer(input_shape=(num_s,))
        #     self.hiddens=[]
        #     for i in hiddens:
        #         self.hiddens.append(tf.keras.layers.Dense(
        #             i, activation='tanh')
        #         )
        #     self.output_layer= tf.keras.layers.Dense(num_a)
        
            self.inImg= tf.keras.layers.InputLayer(input_shape=(20,20,3)) #implies grayscale

            self.inGLOC= tf.keras.layers.InputLayer(input_shape=(1,2))
            self.inUAVLOC= tf.keras.layers.InputLayer(input_shape=(1,2))
            self.inDist=tf.keras.layers.InputLayer(input_shape=(1,))
            
            #img processing, could feed into model... could use for like emeddding, encoding, move to abstract mdp etc
            
            self.xs=[]
            
            self.xs.append(tf.keras.layers.Conv2D(32, (2,2), strides=1) ) 
            self.xs.append(tf.keras.layers.BatchNormalization())
            self.xs.append(tf.keras.layers.MaxPooling2D())

            self.xs.append(tf.keras.layers.Conv2D(64, (3,3)))
            self.xs.append(tf.keras.layers.BatchNormalization())
            self.xs.append(tf.keras.layers.MaxPooling2D())
            self.xs.append(tf.keras.layers.Flatten())


            self.xs.append(tf.keras.layers.Dense(512)) #number is shared dim with other layers, all have to have the same output size
            

            #dont need the model anymore
            #t=tf.keras.Model(x) #versus handing in the input too...?
            #done for now

            #for pos maybe an lstm? should also maybe consisder autoencoders, and muzero uses a resnet I think I can make my own
            
            #need something for pos
            self.ys=[]
            # self.ys.append(tf.keras.layers.Conv1D(1,(1,)))
            # self.ys.append(tf.keras.layers.MaxPool1D())
            # self.ys.append(tf.keras.layers.Flatten())
            self.ys=dense_nn( [512,512])

            self.zs=[]
            # self.zs.append(tf.keras.layers.Conv1D(1,(1,)))
            # self.zs.append(tf.keras.layers.MaxPool1D())
            # self.zs.append(tf.keras.layers.Flatten())
            self.zs=dense_nn([512,512])

            self.ws=dense_nn( [512,512])

            # .output is from keras.model I think, need to move this to call
            

            # f=tf.keras.layers.Dense(1024)(combined)
            # f=tf.keras.layers.Dense(1024)(f)


            self.d1=tf.keras.layers.Dense(512)
            self.d2=tf.keras.layers.Dense(512)

            self.q=tf.keras.layers.Dense(1, name="V_val")

            self.adv=tf.keras.layers.Dense(num_a)

            class test(tf.keras.Model):
                  def __init__(self):
                      super(test,self).__init__()
                      self.x = tf.keras.layers.Dense(1024)
                      print("made test model")

                  @tf.function
                  def call(self, input):
                      return self.x(input)


            self.test_layer= test()



            # # advantage 

            # v_split, adv_split= tf.keras.layers.Lambda(lambda w: tf.keras.split(w, 2,3)) (f)
            
            # v_split=tf.keras.layers.Flatten()(v_split)
            # val_func= tf.keras.layers.Dense(1)(v_split)

            # adv_split=tf.keras.layers.Flatten()(adv_split)
            # adv_split=tf.keras.layers.Dense(num_a)(adv_split)

            # reduce_mean= tf.keras.layers.Lambda( lambda w: tf.reduce_mean(w, axis=1, keepdims=True))

            # q_vals= tf.keras.layers.Add(val_func, reduce_mean(adv_split))

            # #we're subclassing from model... so what're we doing here... Model wont work with the multi path
            # # model=tf.keras.Model(inputs=[t.inputs, y.inputs, z.inputs,w.inputs],outputs=q_vals )

            # # print(model.summary())
            
    #doing this because of keras class model for defn. and because custom loss fn I think
    #and we're doing it so we can do what  ever we want when we call the model 
    #whereas tf.keras.Nodel( <input>) will just follow the transforms
    #so we could maybe do stuff like extract the data from a dict and conver to np before the forward call

    #so here I would need to define the order of layer execution
    # @tf.function
    # def call(self, inputs):
    #     x=self.input_layer(inputs)
    #     for l in self.hiddens:
    #         x=l(x)
    #     out=self.output_layer(x)
    #     return out

    @tf.function
    def call(self,inputs0, inputs1,inputs2, inputs3, value=False):
        img=inputs0
        s_pos=inputs1
        g_pos=inputs2
        dist=inputs3

        x=self.inImg(img)
        for l  in self.xs:
            x=l(x)
        y=self.inUAVLOC(s_pos)
        for l in self.ys:
            y=l(y)
        z=self.inGLOC(g_pos)
        for  l in self.zs:
            z=l(z)
        w=self.inDist(dist)
        for l in self.ws:
            w=l(w)

        c=tf.keras.layers.Lambda(lambda w: tf.keras.layers.concatenate(w))
        combined= c([x,y,z,w])


        #Im making new layers here
        f=self.d1(combined)
        #f=self.d2(f)
        f=self.test_layer(f)
        # advantage 

        v_split, adv_split= tf.keras.layers.Lambda(lambda w: tf.split(w, 2,1)) (f)
            
        v_split=tf.keras.layers.Flatten()(v_split)
        val_func=self.q(v_split)
        # if value== True:
        self.state_value=val_func
        #   return self.state_value
        

        adv_split=tf.keras.layers.Flatten()(adv_split)
        adv_split=self.adv(adv_split)

        reduce_mean= tf.keras.layers.Lambda( lambda w: tf.reduce_mean(w, axis=1, keepdims=True))

        q_vals= tf.keras.layers.Add()([val_func, tf.keras.layers.Subtract()([adv_split,reduce_mean(adv_split)])])

        return q_vals
    #need eager exec or 
    def value_of_most_recent(self):
        return self.state_value

#class value_branch_model(tf.keras.Model):









class dqn:
    def __init__(self,states,actions,hiddens, gamma, max_e_buffer, min_e_buffer, batch_size, lr):
        self.actions=actions
        self.batch_size=batch_size
        self.opt=tf.optimizers.Adam(lr)
        self.gamma=gamma
        self.model= model(states,hiddens,actions)
        self.exp= {'s':[], 'a':[], 'r':[], 's_prime':[], 'done': []}
        self.max_exp= max_e_buffer
        self.min_exp= min_e_buffer
        
    def pred(self,inputs0, inputs1, inputs2, inputs3, value=False):
        try:
          #np.atleast_2d(inputs.astype('float32'))
           
            return self.model(
            np.atleast_2d(inputs0.copy()),
            np.atleast_2d(inputs1.astype('float32')),
            np.atleast_2d(inputs2.astype('float32')),
            np.atleast_2d(inputs3.astype('float32')),
            value=value
            )
        except:
            import matplotlib.pyplot as plt
            from tensorflow.keras.preprocessing.image import array_to_img
            from IPython.display import display
            print("input0")
            print(inputs0.shape)
            display(array_to_img(inputs0[0]))
            #print(inputs0)
            print("input1")
            print(inputs1.shape)
            print(inputs1)
            print("input2")
            print(inputs2.shape)
            print(inputs1)
            print("input3")
            print(inputs3.shape)
            #ins=np.atleast_2d(inputs.astype('float32'))
            return self.model(inputs0,inputs1.astype('float32'),inputs2.astype('float32'),inputs3.astype('float32'))
    
    def train(self, TargetNet):
        if len(self.exp['s'])<self.min_exp:
            return 0

            # 'contract_pos': Box(0,np.Inf, shape=(2,), dtype=np.uint16),
            # 'uav_pos': Box(0,np.Inf, shape=(2,), dtype=np.uint16),
            # 'view': Box(low=0,high=255, shape=(20,20,1), dtype=np.uint8),
            # 'distance': Box(low=-1,high=20000,shape=(1,), dtype=np.int16)
            
        ids= np.random.randint(low=0, high=len(self.exp['s']), size=self.batch_size)
        # print("breaker")
        # print(type(self.exp['s'][0][0]))
        # print(self.exp['s'][0][0].shape)
        
        states0=np.asarray([ self.exp['s'][i][0]  for i in ids] )
        states1=np.asarray([ self.exp['s'][i][1]  for i in ids] )
        states2=np.asarray([ self.exp['s'][i][2]  for i in ids] )
        states3=np.asarray([ self.exp['s'][i][3]  for i in ids] )
        

        
        #[self.exp['s_prime'][i][0],self.exp['s_prime'][i][1], self.exp['s_prime'][i][2],  self.exp['s_prime'][i][3] ]
        actions=np.asarray([self.exp['a'][i] for i in ids])
        rew=np.asarray([self.exp['r'][i] for i in ids])
        #s_prime=np.asarray([ [self.exp['s_prime'][i][0],self.exp['s_prime'][i][1], self.exp['s_prime'][i][2],  self.exp['s_prime'][i][3] ] for i in ids] )
        done=np.asarray([self.exp['done'][i] for i in ids])
        
        states0p=np.asarray([ self.exp['s_prime'][i][0]  for i in ids] )
        states1p=np.asarray([ self.exp['s_prime'][i][1]  for i in ids] )
        states2p=np.asarray([ self.exp['s_prime'][i][2]  for i in ids] )
        states3p=np.asarray([ self.exp['s_prime'][i][3]  for i in ids] )

        #print("value_next call")
        value_next=np.max(TargetNet.pred(states0p,states1p,states2p,states3p), axis=1)
        actual_values=np.where(done, rew, rew+self.gamma*value_next)
        #print("action gradients")
        #print(self.model.test_layer(np.zeros(shape= (1,1024) ) ) )
        with tf.GradientTape() as tape:
            selected_action_values=tf.math.reduce_sum(
                self.pred(states0,states1,states2,states3)*tf.one_hot(actions,self.actions), axis=1
            )
            loss=tf.math.reduce_mean(tf.square(actual_values-selected_action_values))

        variables=self.model.trainable_variables
        gradients= tape.gradient(loss,variables)
        self.opt.apply_gradients(zip(gradients,variables))
        return loss

    def get_action(self,states,epsilon):

        if np.random.random() < epsilon:
            return np.random.choice(self.actions)
        else:
          #np.atleast_2d(states)
            return np.argmax( self.pred(np.asarray([states[0].copy()]),np.asarray([states[1]]) , np.asarray([states[2]]),np.asarray([states[3]]))[0] )
            
    
    def imagine(self,states):
        
        # goal_value=[]
        # for g in goals:
        #     #hack for batch size
        #     g = np.asarray([g ])
        #     goal_value.append(tf.math.reduce_sum(self.pred(g)))

        goal_value = self.pred(np.asarray([states[0].copy()]),np.asarray([states[1]]) , np.asarray([states[2]]),np.asarray([states[3]]), value=True)
        
        
        return goal_value
    
        

    def add_exp(self,exp):
        if len(self.exp['s'])>=self.max_exp:
            for key in self.exp.keys():
                self.exp[key].pop(0)
        for key,val in exp.items():
            self.exp[key].append(val)
    
    def copy_weights(self,train_net):
        var1=self.model.trainable_variables

        var2=train_net.model.trainable_variables
        for v1,v2 in zip(var1,var2):
            v1.assign(v2.numpy())

def transform_obs(obs):
    try:
        if(type(obs) is type({})):
          #obs=obs.copy()
          ret=[]
          ret.append(obs['view'].astype('float32') )
          ret.append(obs['uav_pos'].astype('float32'))
          ret.append(obs['contract_pos'].astype('float32'))
          ret.append(obs['distance'].astype('float32'))
          return np.asarray(ret)
        else:
          return obs
    except:
        print("failed to convert")
        print(obs)


def play(env,train,target, epsilon, copy_step):
        rew=0
        iter=0
        done=False
        obs=env.reset()
        goal=env.reset()
        losses=[]
        rewards=[]


        #m-agent could be for a in agents_ obs:
        while not done:
            obs=transform_obs(obs)
            act=train.get_action(obs,epsilon)
            # print("imagine")
            # print(train.imagine(transform_obs(goal)))

            #act=train.get_action(transform_obs(obs),epsilon)
            prev_obs=obs
            obs,rew,done,_ = env.step(act)

          
            if done:
                #do stuff
                env.reset()#this is it!
            obs=transform_obs(obs)
            prev_obs=transform_obs(prev_obs)
            exp={'s':prev_obs, 'a': act, 'r': rew, 's_prime': obs, 'done': done}
            train.add_exp(exp)
            loss= train.train(target)
            rewards.append(rew)
            


            if isinstance(loss, int):
                losses.append(loss)
            else:
                losses.append(loss.numpy())
            iter+=1

            if iter %copy_step==0:
                #print("updated weights!")
                target.copy_weights(train)
        
        # print("updated weights!")
        # target.copy_weights(train)
        return np.mean(rewards), np.mean(losses)

def main():
    env=uavEnv({})
    gamma=0.99
    copy_step=25
    num_s=len(env.observation_space.sample())
    num_a= env.action_space.n
    hiddens=[200,200]
    max_exp= 10000
    min_exp=5000
    batch_size=128
    lr=1e-5

    train_net = dqn(num_s, num_a, hiddens, gamma, max_exp, min_exp, batch_size, lr)
    target_net = dqn(num_s, num_a, hiddens, gamma, max_exp, min_exp, batch_size, lr)
    

    N=50000
    epsilon=.99

    for n in range(N):
        total_rew, losses = play(env, train_net, target_net, epsilon, copy_step)
        print("reward and loss")
        print(total_rew, losses)

        # env.close()



# if __name__ == '__main__':
#     main()





# inGLOC= tf.keras.layers.Input(shape=(1,1))

# test= dense_nn(inGLOC, [64,64,64])


test_obs=[np.zeros(shape=(1,28,28,3), dtype=np.float32),
np.zeros(shape=(1,2), dtype=np.float32),
np.zeros(shape=(1,2), dtype=np.float32),
np.zeros(shape=(1,1), dtype=np.float32)] 

test_obs0=np.zeros(shape=(1,28,28,3), dtype=np.float32)
test_obs1=np.zeros(shape=(1,2), dtype=np.float32)
test_obs2=np.zeros(shape=(1,2), dtype=np.float32)
test_obs3=np.zeros(shape=(1,1), dtype=np.float32)

# np.array(test_obs)
# test_obs.append([np.zeros(shape=(1,28,28,3), dtype=np.float32),
# np.zeros(shape=(1,2), dtype=np.float32),
# np.zeros(shape=(1,2), dtype=np.float32),
# np.zeros(shape=(1,1), dtype=np.float32)] )

# test_model= model(0,0,4)
# print(test_obs)
# print(test_model(test_obs0,test_obs1,test_obs2,test_obs3 ) )

main()





In rand env
made test model
made test model
percent completed
0.0
collisons
2
time
100
###############
reward and loss
0.020546324015384077 0.0


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Streaming output truncated to the last 5000 lines.
percent completed
0.0
collisons
4
time
100
###############
reward and loss
0.1443111142956401 2230075.5
percent completed
0.0
collisons
6
time
100
###############
reward and loss
0.007440784251348158 2264287.0
percent completed
0.0
collisons
9
time
100
###############
reward and loss
0.007697988948380919 2224359.2
percent completed
0.0
collisons
1
time
100
###############
reward and loss
0.02506124062220305 2156235.8
percent completed
0.0
collisons
1
time
100
###############
reward and loss
0.014954560362599534 2553907.8
percent completed
0.0
collisons
0
time
100
###############
reward and loss
0.05224238982986563 2170600.5
percent completed
0.0
collisons
1
time
100
###############
reward and loss
0.01955393632760841 2489862.5
percent completed
0.0
collisons
1
time
100
###############
reward and loss
0.05787690186427109 2312507.8
percent completed
0.0
collisons
10
time
100
###############
reward and loss
0.008295644313040983 2226998.5


input1
(128, 2)
[[33. 58.]
 [26. 46.]
 [30. 46.]
 [36. 29.]
 [37. 33.]
 [45. 52.]
 [20. 33.]
 [37. 38.]
 [29. 47.]
 [28. 53.]
 [28. 49.]
 [26. 21.]
 [45. 25.]
 [23. 34.]
 [39. 27.]
 [48. 29.]
 [28. 43.]
 [23. 41.]
 [53. 44.]
 [48. 53.]
 [51. 54.]
 [54. 28.]
 [29. 50.]
 [56. 25.]
 [53. 30.]
 [42. 37.]
 [36. 24.]
 [51. 24.]
 [20. 29.]
 [50. 42.]
 [27. 45.]
 [33. 25.]
 [30. 32.]
 [58. 22.]
 [52. 48.]
 [21. 36.]
 [49. 58.]
 [40. 54.]
 [43. 47.]
 [46. 51.]
 [51. 57.]
 [34. 50.]
 [30. 43.]
 [44. 51.]
 [24. 27.]
 [29. 20.]
 [54. 50.]
 [55. 22.]
 [45. 40.]
 [58. 36.]
 [32. 31.]
 [26. 27.]
 [50. 25.]
 [48. 26.]
 [48. 22.]
 [23. 50.]
 [25. 45.]
 [20. 33.]
 [53. 50.]
 [36. 44.]
 [41. 29.]
 [44. 28.]
 [26. 49.]
 [48. 31.]
 [54. 29.]
 [40. 46.]
 [31. 42.]
 [20. 47.]
 [35. 43.]
 [52. 54.]
 [20. 32.]
 [29. 53.]
 [32. 59.]
 [33. 56.]
 [25. 31.]
 [22. 46.]
 [50. 47.]
 [46. 42.]
 [45. 40.]
 [20. 28.]
 [33. 47.]
 [25. 29.]
 [25. 50.]
 [48. 23.]
 [58. 22.]
 [28. 21.]
 [43. 33.]
 [20. 37.]
 [20. 46.]
 [47.

input1
(128, 2)
[[49. 53.]
 [25. 44.]
 [58. 25.]
 [45. 35.]
 [21. 43.]
 [46. 31.]
 [22. 29.]
 [48. 43.]
 [30. 34.]
 [31. 24.]
 [22. 27.]
 [55. 47.]
 [33. 28.]
 [26. 29.]
 [59. 22.]
 [30. 38.]
 [51. 25.]
 [58. 29.]
 [29. 36.]
 [31. 48.]
 [21. 29.]
 [29. 25.]
 [54. 25.]
 [33. 28.]
 [52. 56.]
 [57. 37.]
 [53. 43.]
 [57. 38.]
 [59. 50.]
 [46. 49.]
 [49. 49.]
 [45. 28.]
 [58. 22.]
 [22. 49.]
 [51. 40.]
 [28. 50.]
 [24. 27.]
 [23. 49.]
 [28. 54.]
 [53. 59.]
 [54. 55.]
 [38. 55.]
 [20. 25.]
 [21. 28.]
 [24. 46.]
 [53. 37.]
 [23. 43.]
 [26. 23.]
 [27. 50.]
 [43. 59.]
 [33. 21.]
 [44. 43.]
 [40. 38.]
 [42. 32.]
 [44. 57.]
 [23. 30.]
 [21. 49.]
 [54. 36.]
 [42. 35.]
 [50. 46.]
 [44. 23.]
 [47. 37.]
 [28. 29.]
 [29. 49.]
 [48. 58.]
 [21. 29.]
 [21. 42.]
 [39. 54.]
 [20. 48.]
 [33. 57.]
 [44. 51.]
 [53. 50.]
 [32. 32.]
 [52. 55.]
 [23. 48.]
 [23. 45.]
 [47. 35.]
 [38. 38.]
 [42. 47.]
 [30. 28.]
 [21. 35.]
 [51. 41.]
 [22. 30.]
 [24. 49.]
 [49. 49.]
 [32. 30.]
 [26. 45.]
 [42. 58.]
 [37. 38.]
 [34.

KeyboardInterrupt: ignored

# RAND GRID ENV

In [15]:

import gym
from gym.spaces import Discrete,Box,Dict, Tuple, MultiDiscrete
import numpy as np

from collections import OrderedDict
import tree

MAX_NUM_POINTS=75
MAP_SIZE=80
def clip(x):
        if x<0:
            return 0
        if x>MAP_SIZE:
            return MAP_SIZE
        return x
def pad(img, h, w):
    #  in case when you have odd number
    top_pad = np.floor((h - img.shape[0]) / 2).astype(np.uint16)
    bottom_pad = np.ceil((h - img.shape[0]) / 2).astype(np.uint16)
    right_pad = np.ceil((w - img.shape[1]) / 2).astype(np.uint16)
    left_pad = np.floor((w - img.shape[1]) / 2).astype(np.uint16)
    return np.copy(np.pad(img, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)), mode='constant', constant_values=0))
    
class uavEnv():
    def __init__(self,config,return_agent_actions=False, part=False):
        print("In rand env")
        self.num_agents=1#config['num_uavs']#int(np.random.random()*config['num_uavs'])+2
        self.num_points=1#config['num_points']+int(np.random.random()*15)
        #self.num_agents=2
        self.map_size=MAP_SIZE
        self.reset_amount=0

        self.uav_space= Dict({
            'contract_pos': Box(0,np.Inf, shape=(2,), dtype=np.uint16),
            'uav_pos': Box(0,np.Inf, shape=(2,), dtype=np.uint16),#Box(-100,100, shape=(3,) ),
            # 'contract_status': Repeated(Discrete(4),max_len=MAX_NUM_POINTS),#Repeated(Discrete(3),max_len=(NUM_POINTS)),
            # 'capital': Box(0,100, shape=(1,)),
            # 'owners': Repeated(Discrete(2),max_len=MAX_NUM_POINTS), #Repeated(Discrete(2), max_len=(NUM_POINTS))
            # 'for_sale':Repeated(Tuple( (Discrete(MAX_NUM_POINTS),Discrete(MAX_NUM_AGENTS)) ), max_len=MAX_NUM_POINTS*MAX_NUM_AGENTS)
            'view': Box(low=0,high=255, shape=(20,20,1), dtype=np.uint8),
            'distance': Box(low=-1,high=20000,shape=(1,), dtype=np.int16)
            #'completed': Discrete(MAX_NUM_POINTS)
            #'goal': Tuple( (Discrete(100),Discrete(100)) )
        })
        #self.observation_space=Repeated(self.uav_space,max_len=self.num_agents)
        self.observation_space=self.uav_space#Dict(d)
        # self.action_space= Dict({

        #     'direction': Discrete(4),
        #     #'intension': Discrete(MAX_NUM_POINTS)
        #     # 'bid_for_point': Discrete(MAX_NUM_POINTS+1), #change to select the point  in for_sale list
        #     # 'bid_amount': Box(0,100, shape=(1,)),
        #     # #'bid_accept': Box(0,100,shape=(1,)),
        #     # 'sell_point': Discrete(MAX_NUM_POINTS+1)
        # })
        self.action_space= Discrete(4)
    
    def reset(self):

        self.reset_amount+=1

        contracts={}
        

        self.bids=OrderedDict()
        self.completed=0
        self.capital=25.0
        self.obs=OrderedDict()
        #metrics
        self.t=0
        self.d=0
        self.c=0
        self.d_prev=0
        
        c=[Contract(x=int(np.random.random()*(self.map_size-41)+21),y=int(np.random.random()*(self.map_size-41)+21)) for i in range(self.num_points)]
        refNum=0
        
        for con in c:
            name=refNum
            contracts[con.pos]=con
            refNum+=1
        
        self._decode_to_contract=contracts
        contract_pos=[]
        for val in c: #is this faster than the dict's .items() ? 
            contract_pos.append(val.pos)#np.array(val.pos, dtype=np.float32))
        
        self.contract_pos=contract_pos
        

        w, h = self.map_size, self.map_size
        self.data = np.zeros((h, w, 3), dtype=np.uint8)

        #world border
        self.data[0:20,0:self.map_size]=[255,0,0]
        self.data[self.map_size-20: self.map_size,0:self.map_size ]=[255,0,0]

        self.data[0:self.map_size,0:20]=[255,0,0]
        self.data[0:self.map_size, self.map_size-20: self.map_size ]=[255,0,0]

        #no fly zones : red
        for i in range(200):
            placex=int(np.floor(np.random.random()*w))
            placey=int(np.floor(np.random.random()*h))
            
            self.data[placey, placex]=[255, 0, 0]

        for x,y in contract_pos:
            
            assert x>20 and x<self.map_size-20, print(contract_pos)
            assert y>20 and y<self.map_size-20, print(contract_pos)
            self.data[y, x]=[0, 255, 0]

        k=self.num_agents


        while k>0:
            placex=int(np.floor(np.random.random()*(w-40)+20))
            placey=int(np.floor(np.random.random()*(h-40)+20))
            #print(placex,placey)
            free=True
            # for p in self.data[clip(placex-10):clip(placex+10), clip(placey-10):clip(placey+10) ]:
            #     if p== [255, 0, 0]:
            #         free=False
            # if any(any(any(p) for p in row) for row in self.data[clip(placex-2):clip(placex+2), clip(placey-2):clip(placey+2) ]):
            #     free=False
            if any(self.data[placey,placex]):
                free=False
            if free ==True:
                self.data[placey,placex]=[255,255,255]
                self_view=self.data[(y-10):(y+10), (x-10):(x+10) ].copy()#pad(self.data[clip(placey-10):clip(placey+10), clip(placex-10):clip(placex+10) ].copy(), 20,20)#could be smaller than square, is this an issue?
                self.data[placey,placex]=[0,0,255]
                distance= [(x-placex)**2+(y-placey)**2 for x,y in contract_pos]
                # for i in range(MAX_NUM_POINTS-len(distance)):
                #     distance.append(-1)

                
                self.obs={
                    'uav_pos': np.asarray([placex,placey]),
                    'view': self_view,
                    'contract_pos' :np.asarray(contract_pos,dtype=np.uint16)[0],
                    'distance' : np.asarray(distance)
                   
                }
                
                k-=1
            

        


        
        # import matplotlib.pyplot as plt
        # from tensorflow.keras.preprocessing.image import array_to_img
        # from IPython.display import display
        # display(array_to_img(self.data))
        # print("reset")
        assert self.obs['view'].shape == (20,20,3), print( self.obs['view'].shape ,self.obs['uav_pos'])
        # print(self.observation_space.sample())
        return self.obs


    def _distance(self,x=0 ,y=0,z=0,gx=0,gy=0,gz=0):
        return (x-gx)**2+(y-gy)**2+(z-gz)**2

    def step(self,action):
                obs,rew,done,info={},{},{},{}
                done["__all__"]=False
        # print("action dict")
        # print(action_dict)
                #print("step")
        #try:
              
           
                reward= 0
                point=action
        
                pos = self.obs['uav_pos']
                x=pos[0]
                y=pos[1]
                #z=pos[2], till we add it again
                #clear uav pixel/// is this x,y? check!

                self.data[y,x]= [0,0,0] #can I do this with whats in the ifs down below?
                if point==0 and x<self.map_size:
                    x+=1
                elif point==1 and x>0:
                    x-=1
                elif point==2 and y<self.map_size:
                    y+=1
                elif point==3 and y>0:
                    y-=1
                # elif point==4 and z<99:
                #     z+=1
                # elif point==5 and z>0:
                #     z-=1
                if point in range(5):
                    self.d+=1
                
                distance=[]
                for k,v in self._decode_to_contract.items():
                    if not v._complete:
                        px,py= v.pos
                        distance.append(np.sqrt((x-px)**2+(y-py)**2) )
                        dist=1/(1+distance[0])
                        if dist> self.d_prev:
                            reward+=dist #ONLY FOR 1 POINT !!!!!
                    else:
                        distance.append(-1)
                # for j in range(MAX_NUM_POINTS-len(distance)):
                #     distance.append(-1)
                #print(len(distance))
                self.obs['distance']=np.asarray(distance)
                
                self.d_prev=dist
                #check what everything did, could use functions thats fine.
                color=self.data[y,x] #bigger y is down the screen is that fine?
                if color.tolist() == [0,0,0]:
                    self.obs['uav_pos']=np.asarray( [x,y])
                    self.data[y,x]= [255,255,255] #self color
                    self.obs['view']=self.data[(y-10):(y+10), (x-10):(x+10) ].copy()#pad(self.data[clip(y-10):clip(y+10), clip(x-10):clip(x+10) ],20,20)
                    self.data[y,x]= [0,0,255] #std uav color

                     #blank spot
                elif color.tolist()== [255,0,0]:
                    reward-=.1 #no fly zone
                    self.c+=1
                elif color.tolist()== [0,255,0]:
                    #found POI carry out contract stuff
                    # for j in range(self.num_agents):
                    #     if i != j:
                    #         rew[j]-=.1
                    reward+=25

                    self.data[y,x]=[0,100,0] #completed point color.
                    # if self.obs[i]['contract_pos'][intention] == (x,y):
                    #     reward+=10
                    # self.obs[i]['completed']+=1
                    # reward+=self.obs[i]['completed']*.1
                    # if self._decode_to_contract[(x,y)].stakeHolders == i:
                    #     reward+=1
                    self.completed+=1
                    done["__all__"]=True
                    
                elif color.tolist() == [0,0,255]:
                    reward-=.1 #another uav is there
                    self.c+=1
                    # print("num uavs", self.num_agents)
                    # print("x,y", x, y)

                    
                

                #clean up in case it was an invalid move
                pos = self.obs['uav_pos']
                x=pos[0]
                y=pos[1]
                self.data[y,x]=[0,0,255]
                
                #self.obs[i]['uav_pos']= (x,y)# untill we addd z in again,z)#np.asarray([x,y,z])
                

                # alpha=.1
                # beta=100
                # for key, val in self._decode_to_contract.items():
                #     if val._complete==False:
                #         if val.complete(uav_id,x,y) : #need to put contract data where true is, and update the net's representation of what's been traveled to   
                #             reward+=val.timeBonus()
                #             self.completed+=1
                #             # for j in range(self.num_agents):
                #             #     self.obs[j]['contract_status'][key]=1 #the action is the point number, 1 is flag for completed, could make into the uav's id and save one of the conditions in the next set of if's
                        

                # for key,val in self._decode_to_contract.items():
                #     if self.obs[i]['owners'][key] ==1:
                #         if self._distance(x=x,y=y,gx=val.pos[0],gy=val.pos[1]) != 0:
                #             reward+=alpha/self._distance(x=x,y=y,gx=val.pos[0],gy=val.pos[1]) + capital_incentive*self.obs[uav_id]['capital'][0]# maybe the reciprocal of the distance, and maybe make that negative
                #         else: capital_incentive*self.obs[uav_id]['capital']
                #     #check if someone else completed the contract
                #     if self.obs[i]['owners'][key]==1 and uav_id != val.completed_by and self.obs[i]['contract_status'][key]==1:
                #         #self._state['contract_status'][point]=1

                #         self.obs[i]['contract_status'][key]=3 #flag for inacive

                #         #print("completed")
                        
                #         reward-=beta #if 

                # print(type(reward))
                # assert type(reward) is float
               
                # print("reward")
                # print(reward[0])

                #done[i]
           
            
                
                if(self.completed==1):
                          #done["__all__"]=True
                          print("got it")
                
                
                self.t+=1
                if(self.t == 100):#supposing they have a velocity of 5 m/s and a timestep 1/5 of a second (they have unit velocity)
                    done["__all__"]=True

                if done["__all__"]==True:
                    print("percent completed")
                    print(self.completed/self.num_points*100)
                    print("collisons")
                    print(self.c)
                    print("time")
                    print(self.t)
                    print("###############")
                
                assert self.obs['view'].shape == (20,20,3),  self.obs['view'].shape
                #else: done["__all__"]= False
                #print("step", self.t)
                # if self.t%3==0: #casts expire, need to let everyone see/ bid
                #     self.buy_broadcasts={}
                #     self.sell_broadcasts={}
                return self.obs, reward,done["__all__"] , info
        # except RuntimeError:
        #     print("action dict error")
        #     print (action_dict)
        #     print("exception")
        #     print(Exception)
            
            
            



        


        

# Contract

In [2]:

import time
def distanceSquared(x,y,gx,gy=0,z=0,gz=0):
        return (x-gx)**2 + (y-gy)**2 + (z-gz)**2
def within(x,x1):
    if(x1-2 <x < x1+2):
        return True
    return False

class Contract:

    def __init__(self,x=0,y=0,z=0,price=50, timeBonus=10):
        K_RES=4
        self.price=price
        self.pos= (x,y)
        self.timebonus=timeBonus
        self.startTime=time.time()
        self.stakeHolders=None#unused as the enviroment checks for ownership using numbers assoiated with UAV id, and -1 as a flag for inactive. 
        self._complete=False
        self.completed_by= None
        self.size=2
        self.k=K_RES
        self.visits=0

    def complete(self,uav_id,x,y,z=0,r=1):
        # if (distanceSquared(x,y,self.pos[0],self.pos[1],z,self.pos[2]) < r**2):
        #     self.completed_by=uav_id
        #     self._complete=True
        #     return True
        # else: 
        #     return False
        # if(within(x,self.pos[0]) and within(y,self.pos[1])):
        #     self.completed_by=uav_id
        #     self._complete=True
        #     return True
        # else:
        #     return False
        if x==self.pos[0] and y==self.pos[1]:
            self.completed_by=uav_id
            self._complete=True
            self.visits+=1
            self.ticks=20
            return True
        else:
            return False

    def timeBonus(self):
        return self.timebonus - (self.elapsedTime())/10000

    def elapsedTime(self):
        return time.time()-self.startTime